In [1]:
import pandas

In [2]:
df = pandas.read_csv("metrics.csv", names=["basename", "seed", "i", "PSNR", "SSIM", "LPIPS"])

temp = df["basename"].str.split("_", n=2, expand=True)
temp[0] = temp[0] + " " + temp[1]
temp[2] = temp[2].replace("_", " ", regex=True).str.upper()
temp = temp[[0, 2]]

df[["task", "method"]] = temp
df = df.drop(["basename", "seed", "i"], axis="columns")

In [3]:
mean = df.groupby(["task", "method"]).mean().reset_index().round(2)
mean = mean.pivot(index="method", columns="task", values=["PSNR", "SSIM", "LPIPS"])

rank = mean.rank(method="dense", ascending=False)
rank["LPIPS"] = mean["LPIPS"].rank(method="dense")

text = mean.map('{:.2f}'.format)
bold = r"\textbf{" + text + "}"
line = r"\underline{" + text + "}"

text = bold.where(rank == 1, text)
text = line.where(rank == 2, text)
text = text.swaplevel(0, 1, axis=1).sort_index(axis=1)

print(text.to_latex())

\begin{tabular}{lllllllllllll}
\toprule
task & \multicolumn{3}{r}{inpainting box} & \multicolumn{3}{r}{inpainting random} & \multicolumn{3}{r}{motion deblur} & \multicolumn{3}{r}{super resolution} \\
 & LPIPS & PSNR & SSIM & LPIPS & PSNR & SSIM & LPIPS & PSNR & SSIM & LPIPS & PSNR & SSIM \\
method &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
DPS 0100 & 0.38 & 16.97 & 0.51 & 0.39 & 16.84 & 0.50 & 0.30 & 19.84 & 0.58 & 0.35 & 18.40 & 0.55 \\
DPS 1000 & 0.22 & 21.26 & 0.65 & 0.20 & 21.92 & 0.66 & 0.17 & 22.96 & 0.66 & 0.16 & 25.06 & 0.72 \\
MMPS 0100 01 & \textbf{0.19} & \underline{21.57} & \textbf{0.68} & 0.19 & 22.20 & 0.69 & 0.20 & 24.00 & 0.71 & 0.15 & \textbf{27.32} & \textbf{0.79} \\
MMPS 0100 05 & \underline{0.20} & \textbf{21.65} & \underline{0.67} & \underline{0.15} & \textbf{22.89} & \textbf{0.72} & \underline{0.13} & \textbf{26.74} & \textbf{0.77} & 0.13 & \underline{27.11} & \underline{0.78} \\
MMPS 1000 01 & \textbf{0.19} & 21.11 & 0.65 & 0.18 & 21.96 & 0.66 & 0.16 & 23.93